<a href="https://colab.research.google.com/github/angiequima/Econometria_QLab_PC/blob/main/PC_3/PC3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Práctica Calificada 3**

Sobre la base de los datos y lo visto en clases, responda lo siguiente (2 puntos cada una):

1. Realice una descripción de las características de los individuos de la muestra.

In [2]:
# Instalar e importar las librerías necesarias
!pip install statsmodels pandas linearmodels scipy
!pip install --upgrade linearmodels

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.3/117.3 kB 9.6 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import scipy.stats as stats

from linearmodels import OLS
from linearmodels.iv import IV2SLS, compare
from statsmodels.api import add_constant
from linearmodels.iv import IVGMM
from scipy import stats

In [4]:
# Explorar el archivo .dta

data = pd.read_stata("card.dta")
data = add_constant(data, has_constant="add") # Agregar una constante que servirá de intercepto

data.head()

,const,id,nearc2,nearc4,educ,age,fatheduc,motheduc,weight,momdad14,...,smsa66,wage,enroll,KWW,IQ,married,libcrd14,exper,lwage,expersq
0,1.0,2,0,0,7,29,NaN,NaN,158413.0,1,...,1,548,0,15.0,NaN,1.0,0.0,16,6.306275,256
1,1.0,3,0,0,12,27,8.0,8.0,380166.0,1,...,1,481,0,35.0,93.0,1.0,1.0,9,6.175867,81
2,1.0,4,0,0,12,34,14.0,12.0,367470.0,1,...,1,721,0,42.0,103.0,1.0,1.0,16,6.580639,256
3,1.0,5,1,1,11,27,11.0,12.0,380166.0,1,...,1,250,0,25.0,88.0,1.0,1.0,10,5.521461,100
4,1.0,6,1,1,12,34,8.0,7.0,367470.0,1,...,1,729,0,34.0,108.0,1.0,0.0,16,6.591674,256


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3010 entries, 0 to 3009
Data columns (total 35 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   const     3010 non-null   float64
 1   id        3010 non-null   int16  
 2   nearc2    3010 non-null   int8   
 3   nearc4    3010 non-null   int8   
 4   educ      3010 non-null   int8   
 5   age       3010 non-null   int8   
 6   fatheduc  2320 non-null   float64
 7   motheduc  2657 non-null   float64
 8   weight    3010 non-null   float32
 9   momdad14  3010 non-null   int8   
 10  sinmom14  3010 non-null   int8   
 11  step14    3010 non-null   int8   
 12  reg661    3010 non-null   int8   
 13  reg662    3010 non-null   int8   
 14  reg663    3010 non-null   int8   
 15  reg664    3010 non-null   int8   
 16  reg665    3010 non-null   int8   
 17  reg666    3010 non-null   int8   
 18  reg667    3010 non-null   int8   
 19  reg668    3010 non-null   int8   
 20  reg669    3010 non-null   int8

**Descripción de Variables**

*   **const**: Variable constante añadida para el intercepto en modelos de regresión.
*   **id**: Identificador único para cada individuo.
*   **nearc2**: Indicador si el individuo vivió cerca de un colegio de 2 años (0=no, 1=sí).
*   **nearc4**: Indicador si el individuo vivió cerca de un colegio de 4 años (0=no, 1=sí).
*   **educ**: Años de educación completados.
*   **age**: Edad del individuo.
*   **fatheduc**: Años de educación del padre.
*   **motheduc**: Años de educación de la madre.
*   **weight**: Peso de la encuesta (para ajustes muestrales).
*   **momdad14**: Indicador si ambos padres vivían en casa cuando el individuo tenía 14 años.
*   **sinmom14**: Indicador si solo la madre vivía en casa cuando el individuo tenía 14 años.
*   **step14**: Indicador si un padrastro vivía en casa cuando el individuo tenía 14 años.
*   **reg661** a **reg669**: Variables dummy para la región de residencia en 1966.
*   **south66**: Indicador si el individuo vivía en el sur en 1966.
*   **black**: Indicador si el individuo es afroamericano (0=no, 1=sí).
*   **smsa**: Indicador si el individuo vivía en un Área Estadística Metropolitana (SMSA) en el momento de la encuesta.
*   **south**: Indicador si el individuo vivía en el sur en el momento de la encuesta.
*   **smsa66**: Indicador si el individuo vivía en un SMSA en 1966.
*   **wage**: Salario por hora.
*   **enroll**: Indicador si el individuo estaba matriculado en la escuela en el momento de la encuesta.
*   **KWW**: Puntuación en la prueba de conocimiento del mundo del trabajo (Knowledge of the World of Work).
*   **IQ**: Coeficiente intelectual (IQ).
*   **married**: Indicador si el individuo está casado (0=no, 1=sí).
*   **libcrd14**: Número de libros en la biblioteca de casa cuando el individuo tenía 14 años.
*   **exper**: Experiencia laboral en años (age - educ - 6).
*   **lwage**: Logaritmo natural del salario por hora (`wage`).
*   **expersq**: Experiencia laboral al cuadrado (`exper`^2).

In [6]:
summary = data.describe()
print(summary)

        const           id       nearc2       nearc4         educ  \
count  3010.0  3010.000000  3010.000000  3010.000000  3010.000000   
mean      1.0  2581.748837     0.440864     0.682060    13.263455   
std       0.0  1500.538849     0.496573     0.465753     2.676913   
min       1.0     2.000000     0.000000     0.000000     1.000000   
25%       1.0  1275.500000     0.000000     0.000000    12.000000   
50%       1.0  2541.000000     0.000000     1.000000    13.000000   
75%       1.0  3858.750000     1.000000     1.000000    16.000000   
max       1.0  5225.000000     1.000000     1.000000    18.000000   

               age     fatheduc     motheduc        weight     momdad14  ...  \
count  3010.000000  2320.000000  2657.000000  3.010000e+03  3010.000000  ...   
mean     28.119601    10.003448    10.348137  3.211852e+05     0.789369  ...   
std       3.137004     3.720737     3.179671  1.706459e+05     0.407825  ...   
min      24.000000     0.000000     0.000000  7.560700e+04

In [9]:
# Seleccionar el grupo de variables relevantes en el paper

vars_desc = ["lwage", "educ", "exper", "black", "nearc4"]
data[vars_desc].describe()

,lwage,educ,exper,black,nearc4
count,3010.000000,3010.000000,3010.000000,3010.000000,3010.000000
mean,6.261832,13.263455,8.856146,0.233555,0.682060
std,0.443797,2.676913,4.141672,0.423162,0.465753
min,4.605170,1.000000,0.000000,0.000000,0.000000
25%,5.976985,12.000000,6.000000,0.000000,0.000000
50%,6.286928,13.000000,8.000000,0.000000,1.000000
75%,6.563503,16.000000,11.000000,0.000000,1.000000
max,7.784889,18.000000,23.000000,1.000000,1.000000


**¿Qué información nos provee el promedio de la dependiente en la interpretación de los coeficientes?**

El promedio de la variable dependiente, que en este caso es lwage o logaritmo del salario, es **6.26** (o **577.28** dólares), mientras que el promedio de los años de educación en la muestra es de aproximadamente **13.26** años.

El hecho de que la variable dependiente esté en logaritmos permite que los coeficientes se interpreten como *cambios porcentuales* sobre el salario promedio. Conocer el salario promedio permite traducir esos porcentajes en variaciones monetarias.

**¿Cómo se interpretan los coeficientes en este caso?**

En este estudio la variable dependiente es el logaritmo del salario (lwage). Cuando la variable dependiente está en logaritmos y las variables explicativas están en niveles, el modelo es de tipo log-lineal. En este caso, los coeficientes se interpretan como variaciones porcentuales aproximadas en el salario ante un cambio unitario en la variable explicativa.

En particular, el coeficiente asociado a los años de educación (educ) mide el retorno porcentual a un año adicional de escolaridad. Es decir, indica en cuánto porcentaje aumenta el salario cuando el individuo completa un año adicional de educación, manteniendo constantes las demás variables del modelo.

Por ejemplo, si el coeficiente estimado fuese 0.08, esto implicaría que un año adicional de educación está asociado con un incremento aproximado de 8% en el salario.

2. Regresione usando **OLS**

In [16]:
ols_model = smf.ols(
    'lwage ~ educ + exper + expersq + age + black + south + smsa + smsa66 + reg661 + reg662 + reg663 + reg664 + reg665 + reg666 + reg667 + reg668',
    data=data
).fit()

print(ols_model.summary())

                            OLS Regression Results                            
Dep. Variable:                  lwage   R-squared:                       0.300
Model:                            OLS   Adj. R-squared:                  0.296
Method:                 Least Squares   F-statistic:                     85.48
Date:                Tue, 17 Feb 2026   Prob (F-statistic):          1.74e-218
Time:                        20:46:29   Log-Likelihood:                -1288.8
No. Observations:                3010   AIC:                             2610.
Df Residuals:                    2994   BIC:                             2706.
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.3400      0.007     51.778      0.0

**¿Qué ocurre con edad? ¿Por qué?**

Según el modelo, el coeficiente de `age` es 0.7332, significativo y positivo. Se espera que age capture la misma variación que `exper` y `educ`. Sin embargo, existe un problema de multicolinealidad muy fuerte, ya que los coeficientes de `exper` y `educ` son negativos, lo cual es económicamente absurdo.

Al quitar `age` del modelo se elimina la relación lineal casi perfecta entre `educ`, `exp` y `age`. Ahora, los coeficientes de educación y experiencia son positivos y tienen una interpretación económica.


In [17]:
# Quitamos edad del modelo
ols_model = smf.ols(
    'lwage ~ educ + exper + expersq + black + south + smsa + smsa66 + reg661 + reg662 + reg663 + reg664 + reg665 + reg666 + reg667 + reg668',
    data=data
).fit()

print(ols_model.summary())

                            OLS Regression Results                            
Dep. Variable:                  lwage   R-squared:                       0.300
Model:                            OLS   Adj. R-squared:                  0.296
Method:                 Least Squares   F-statistic:                     85.48
Date:                Tue, 17 Feb 2026   Prob (F-statistic):          1.74e-218
Time:                        21:02:21   Log-Likelihood:                -1288.8
No. Observations:                3010   AIC:                             2610.
Df Residuals:                    2994   BIC:                             2706.
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      4.7394      0.072     66.259      0.0

**Interpretación del estimado obtenido para educación:**

Un año adicional de educación aumenta el salario en aproximadamente 7.47%, manteniendo constantes experiencia, región y demás controles.

**Comparación con la Tabla 2, Columna 2, del paper:**

El coeficiente estimado del modelo es muy similar al reportado en la Tabla 2, Columna 2 del paper, donde el retorno a la educación es 7.4% por año adicional de escolaridad. Esto confirma que la especificación replicada es consistente con los resultados originales del estudio.

3. Vea la Tabla 2 de Card (1993).

**¿Qué se encuentra al inspeccionar el coeficiente estimado de educación? ¿Es creíble este coeficiente? ¿Por qué?**

En la Tabla 2 del paper de David Card (1993), el coeficiente OLS de educación es aproximadamente 0.07. Ese nivel es admisible, pero no necesariamente causal porque la educación puede estar correlacionada con componentes no observables, como la habilidad, que también tienen un efecto sobre el salario. En ese sentido, el estimador OLS es inconsistente:
$$
Cov(educ, u) \neq 0
$$



**¿Qué 2 posibles fuentes de inconsistencia podrían ocurrir como resultado de la omisión de variables?**

**a) Sesgo por habilidad (Ability Bias):** La habilidad es una variable no observada que permite obtener salarios más altos, sin importar el nivel educativo. Si un individuo con habilidad consigue más años de escolaridad e ingresos que el promedio, entonces el sesgo será positivo.

$$
\text{Inconsistencia} = \gamma \frac{\text{Cov}(educ,a)}{\text{Var}(educ)}
$$

* $a$: habilidad cognitiva no observada
* $\gamma > 0$ porque a mayor habilidad, mayor salario
* $\text{Cov}(educ, a) > 0$ porque las personas más hábiles estudian más

**b) Impaciencia:** Algunas personas son impacientes y prefieren integrarse al mercado laboral en lugar de continuar con sus estudios. Esta variable tiene guarda una relación negativa con los años de escolaridad y el salario, ya que una mayor impaciencia implica menos años de escolaridad.

$$
\text{Inconsistencia} = \gamma \frac{\text{Cov}(educ,a)}{\text{Var}(educ)}
$$

* $a$: impaciencia por trabajar
* $\gamma > 0$ porque acumula experiencia laboral y, por tanto, su valor en el mercado
* $\text{Cov}(educ, a) < 0$ porque a más impaciencia, menos educación

Finalmente, el sesgo será negativo.


